# Resampling with SimpleITK

This notebook demonstrates how to resample a NIfTI image using SimpleITK. Resampling is a crucial preprocessing step to ensure that all images in a dataset have the same voxel spacing.

## 1. Import Libraries

Let's start by importing the necessary libraries. We'll need `SimpleITK` for resampling and `os` for handling file paths.

In [2]:
import SimpleITK as sitk
import os

## 2. Define the Resampling Function

We'll create a function that takes a SimpleITK image and a new spacing as input. It will then use the `Resample` filter to change the image's voxel spacing. We'll use linear interpolation for this process.

### 2.1 Calculate Resampling Template

This function computes a template image that defines the target grid for resampling. It takes two images (e.g., CT and PET), finds a bounding box that encompasses both in physical space, and creates a new reference image with the desired target spacing. This ensures both modalities are aligned perfectly after resampling.

In [3]:
def calculate_resample_template(ct_image, pet_image, target_spacing):
    """
    Computes a template image for resampling based on the physical space of CT and PET images.

    :param ct_image: SimpleITK image of the CT scan.
    :param pet_image: SimpleITK image of the PET scan.
    :param target_spacing: The desired target spacing as a list or tuple.
    :return: A SimpleITK image to be used as a resampling template.
    """
    # Use numpy for vector operations
    import numpy as np

    # Get physical boundaries of CT image
    ct_origin = np.array(ct_image.GetOrigin())
    ct_size = np.array(ct_image.GetSize())
    ct_spacing = np.array(ct_image.GetSpacing())
    ct_end = ct_origin + (ct_size - 1) * ct_spacing

    # Get physical boundaries of PET image
    pet_origin = np.array(pet_image.GetOrigin())
    pet_size = np.array(pet_image.GetSize())
    pet_spacing = np.array(pet_image.GetSpacing())
    pet_end = pet_origin + (pet_size - 1) * pet_spacing

    # Determine the combined bounding box in physical space
    combined_origin = np.minimum(ct_origin, pet_origin)
    combined_end = np.maximum(ct_end, pet_end)

    # Calculate the size of the new template image
    new_size = np.ceil((combined_end - combined_origin) / target_spacing).astype(int)

    # Create the template image
    template_image = sitk.Image([int(s) for s in new_size], sitk.sitkFloat64)
    template_image.SetOrigin(combined_origin.tolist())
    template_image.SetSpacing(target_spacing)
    # Assume both images have the same orientation
    template_image.SetDirection(ct_image.GetDirection())

    return template_image

In [4]:
def resample_image(image: sitk.Image, 
                           template_image: sitk.Image, 
                           default_value: float, 
                           interpolator: int = sitk.sitkLinear) -> sitk.Image:
    """
    Resamples an image to match the grid of a template image with a specific default value.

    :param image: The SimpleITK image to resample.
    :param template_image: A SimpleITK image defining the target grid.
    :param default_value: The pixel value to use for regions outside the original image.
    :param interpolator: The interpolator to use (e.g., sitk.sitkLinear, sitk.sitkNearestNeighbor).
    :return: The resampled SimpleITK image.
    """
    resampler = sitk.ResampleImageFilter()
    resampler.SetReferenceImage(template_image)
    resampler.SetInterpolator(interpolator)
    resampler.SetDefaultPixelValue(default_value) 

    return resampler.Execute(image)

## 3. Random Process 5 subjects to test function

In [5]:
import random
from tqdm import tqdm
from pathlib import Path
import SimpleITK as sitk


def process_subject(subject_dir: Path, output_dir: Path, target_spacing: list):
    """
    Processes a single subject: loads, resamples, and saves CT, PET, and segmentation images.
    """
    subject_id = subject_dir.name
    
    # 1. 定义并检查所有文件路径
    path_map = {
        "ct": subject_dir / f"{subject_id}_ct.nii.gz",
        "pet": subject_dir / f"{subject_id}_pet.nii.gz",
        "seg_lung": subject_dir / f"{subject_id}_seg-lung.nii.gz",
        "seg_lesion": subject_dir / f"{subject_id}_seg-lesion.nii.gz",
        "seg_lesion_refined": subject_dir / f"{subject_id}_seg-lesion-refined.nii.gz",
    }

    # 检查核心文件是否存在，如果不存在则跳过
    if not path_map["ct"].exists() or not path_map["pet"].exists():
        print(f"Warning: Skipping {subject_id} due to missing CT or PET.")
        return

    # 2. 读取所有存在的图像
    images = {}
    for key, path in path_map.items():
        if path.exists():
            images[key] = sitk.ReadImage(str(path)) # sitk 需要字符串路径
        else:
            print(f"Info: File not found for {subject_id}: {path.name}")
    
    # 3. 计算重采样模板 (必须有CT和PET)
    template_image = calculate_resample_template(images["ct"], images["pet"], target_spacing)

    # 4. 执行重采样
    resampled_images = {}
    resampled_images["ct"] = resample_image(images["ct"], template_image, default_value=-1024, interpolator=sitk.sitkLinear)
    resampled_images["pet"] = resample_image(images["pet"], template_image, default_value=0, interpolator=sitk.sitkLinear)
    
    # 对所有存在的分割掩码进行重采样
    for key in ["seg_lung", "seg_lesion", "seg_lesion_refined"]:
        if key in images:
            resampled_images[key] = resample_image(images[key], template_image, default_value=0, interpolator=sitk.sitkNearestNeighbor)

    # 5. 保存结果
    subject_output_dir = output_dir / subject_id
    subject_output_dir.mkdir(exist_ok=True, parents=True)
    
    for key, img in resampled_images.items():
        output_filename = path_map[key].name # 保持原始文件名
        sitk.WriteImage(img, str(subject_output_dir / output_filename))

In [6]:
target_spacing = [1.5, 1.5, 3.0]
    
# 使用 pathlib 进行路径管理
project_root = Path.cwd().resolve().parents[1]
source_dir = project_root / 'data' / '0_nifti'
output_dir_test = project_root / 'outputs' / 'temp_resample_test'
output_dir_test.mkdir(exist_ok=True, parents=True)

all_subject_dirs = list(source_dir.glob("sub-*/"))

random.seed(42)
sample_dirs = random.sample(all_subject_dirs, 5)

print("随机抽取的5个病例进行测试:")
for d in sample_dirs:
    print(f"- {d.name}")

with tqdm(sample_dirs, desc="Processing subjects") as pbar:
    for subject_dir in pbar:
        pbar.set_description(f"Processing {subject_dir.name}")
        try:
            process_subject(subject_dir, output_dir_test, target_spacing)
        except Exception as e:
            print(f"FATAL ERROR processing {subject_dir.name}: {e}")

随机抽取的5个病例进行测试:
- sub-AKHFRHWALDELFRIEDE20190319
- sub-AKHRETTICHMONIKA20130930
- sub-NM247
- sub-NM042
- sub-AKHDOPPELREITERKARL20230123


Processing sub-AKHFRHWALDELFRIEDE20190319:   0%|          | 0/5 [00:00<?, ?it/s]

Processing sub-AKHDOPPELREITERKARL20230123: 100%|██████████| 5/5 [01:10<00:00, 14.03s/it]


## Crop
add crop function, use seg-lung as bbox to crop

In [ ]:
def crop_to_bounding_box(image: sitk.Image, bounding_box: tuple) -> sitk.Image:
    """
    Crops an image to a given bounding box.

    :param image: The SimpleITK image to crop.
    :param bounding_box: A tuple containing the start indices and size of the box
                        (x_start, y_start, z_start, x_size, y_size, z_size).
    :return: The cropped SimpleITK image.
    """
    # The bounding box from LabelShapeStatisticsImageFilter is in the format:
    # (x_start, y_start, z_start, x_size, y_size, z_size)
    # sitk.RegionOfInterest requires two tuples: size and index.
    start_index = bounding_box[0:3]
    size = bounding_box[3:6]
    
    return sitk.RegionOfInterest(image, size, start_index)

### Get Bounding Box from Mask

This function calculates the bounding box of the largest connected component in a binary mask. It's useful for finding the region of interest, like the lungs, and ignoring smaller, irrelevant artifacts.

In [ ]:
def get_bounding_box_from_mask(mask: sitk.Image) -> tuple:
    """
    Calculates the bounding box of the largest connected component in a binary mask.

    :param mask: The binary SimpleITK mask image.
    :return: A tuple representing the bounding box (x_start, y_start, z_start, x_size, y_size, z_size).
    """
    # Ensure the mask is of an unsigned integer type for connected component analysis
    if mask.GetPixelID() not in [sitk.sitkUInt8, sitk.sitkUInt16, sitk.sitkUInt32, sitk.sitkUInt64]:
        mask = sitk.Cast(mask, sitk.sitkUInt8)

    # Find all connected components in the mask
    cc_filter = sitk.ConnectedComponentImageFilter()
    labeled_mask = cc_filter.Execute(mask)
    
    # Get statistics for each label
    label_stats = sitk.LabelShapeStatisticsImageFilter()
    label_stats.Execute(labeled_mask)
    
    # Find the label of the largest component (excluding background label 0)
    labels = label_stats.GetLabels()
    if 0 in labels:
        labels.remove(0)
    
    if not labels:
        raise ValueError("No foreground objects found in the mask to calculate bounding box.")

    largest_label = max(labels, key=label_stats.GetPhysicalSize)
    
    # Get the bounding box of the largest component
    bounding_box = label_stats.GetBoundingBox(largest_label)
    
    return bounding_box